In [95]:
# ! pip install -q python-dotenv
# ! pip install google-generativeai

from dotenv import load_dotenv
import os
import google.generativeai as genai
import re
import json
import requests

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY") 

genai.configure(api_key=google_api_key)
llm = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
put_code_file_name = input("Enter your code file name: ")
print("Code file name: ", put_code_file_name)

code_number = re.search(r'\d+', put_code_file_name).group()
print("Extracted code number: ", code_number)

Code number:  put_code_2.py
Extracted code number:  2


In [97]:
code_path = f"./PUTs/{put_code_file_name}"

with open(code_path, 'r') as f:
    put_code_content = f.read()

print(put_code_content)

def function_put(a, b, c) -> str:
    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):
        raise TypeError("All inputs must be integers (bool is not allowed).")
    elif not all(x > 0 for x in [a, b, c]):
        raise ValueError("All sides must be greater than zero.")
    elif a + b <= c or a + c <= b or b + c <= a:
        return "Not a valid triangle"
    elif a == b == c:
        return "Equilateral"
    elif a == b or b == c or a == c:
        return "Isosceles"
    return "Scalene"


In [98]:
goals_prompt_f1 = f"""
You are an expert Python software engineer.

Write a complete set of Python `unittest` test cases for the following code.

Your goals:  
- Combine all existing test cases into a single method called `test_all_cases`.
- Use meaningful `assertEqual`, `assertTrue`, `assertFalse`, etc.  
- Output valid Python code only — no explanations.
- All test cases should be in a def test function only.
- For now, generate only 1 or 2 test cases — do not try to cover all edge cases yet.
- In Output, do not include put_code_content or any other text. Just the test cases.
Here is the code you have to write tests for:

```python
{put_code_content}

Example 1:
Code to write test cases for:

```python
def add(a, b):
    return a + b

"""

formatted_prompt = goals_prompt_f1.format()
# print(formatted_prompt)

test_cases = llm.generate_content(formatted_prompt)

print(test_cases.text)

```python
import unittest

class TestFunctionPut(unittest.TestCase):
    def test_all_cases(self):
        self.assertEqual(function_put(2, 2, 2), "Equilateral")
        self.assertEqual(function_put(3, 4, 5), "Scalene")

```



In [99]:
test_cases_str = test_cases.text.strip()
test_cases_str = test_cases_str.replace("```python", "").replace("```", "").strip()

test_suit_path = f"./Final_Test_Suits_Few_shot/test_put_code_{put_code_file_name}"

with open(test_suit_path, 'w') as f:   
    f.write(put_code_content)
    f.write("\n")
    f.write(test_cases_str)  
 

In [100]:
test_suit_path = f"./Final_Test_Suits_Few_shot/test_put_code_{put_code_file_name}"

with open(test_suit_path, 'r') as f:
    test_cases_content = f.read()
print(test_cases_content)

 

def function_put(a, b, c) -> str:
    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):
        raise TypeError("All inputs must be integers (bool is not allowed).")
    elif not all(x > 0 for x in [a, b, c]):
        raise ValueError("All sides must be greater than zero.")
    elif a + b <= c or a + c <= b or b + c <= a:
        return "Not a valid triangle"
    elif a == b == c:
        return "Equilateral"
    elif a == b or b == c or a == c:
        return "Isosceles"
    return "Scalene"
import unittest

class TestFunctionPut(unittest.TestCase):
    def test_all_cases(self):
        self.assertEqual(function_put(2, 2, 2), "Equilateral")
        self.assertEqual(function_put(3, 4, 5), "Scalene")


In [101]:
import ast  
import ast

def LLMC(formatted_prompt):
    test_cases = llm.generate_content(formatted_prompt)
    return test_cases.text

def SyntaxFixer(raw_IUT):
    try:
        ast.parse(raw_IUT)
        return raw_IUT
    except SyntaxError as e:
        syntax_fixed_prompt = f"""
            You are an expert Python developer.The following code is a Python unit test, but it contains syntax errors. Please identify and fix the syntax errors in the code below. After fixing, return only the corrected version of the code.
            Here is the code with syntax errors:
            {raw_IUT}
            Please fix the code and return the corrected version.
            """
        syntax_fixed_prompt = syntax_fixed_prompt.format()
        syntax_fixed_IUT = LLMC(syntax_fixed_prompt)
        # print(syntax_fixed_IUT)
        syntax_fixed_IUT = SyntaxCheck(syntax_fixed_IUT)
        return syntax_fixed_IUT


def SyntaxCheck(syntax_fixed_IUT):
    try:
        ast.parse(syntax_fixed_IUT)
        print("Syntax is correct.")
        # print(syntax_fixed_IUT)
        return syntax_fixed_IUT
    except SyntaxError as e:
        print("Syntax error found.")
        print(e)
        error_line = syntax_fixed_IUT.splitlines()[0]
        return SyntaxCheck(syntax_fixed_IUT.replace(error_line, "", 1))
 

def IntendedBehaviorFixer(syntax_fixed_IUT, PUT):
    fixed_IUT = []
    for test_case in syntax_fixed_IUT:
        expected_output = PUT(*test_case['input'])
        if expected_output != test_case['output']:
            test_case['output'] = expected_output
            fixed_IUT.append(test_case)
    return fixed_IUT


In [102]:
def fix_and_generate_tests(test_cases_content, put_code_content, put_code_file_name):
    
    fixed_syntax = SyntaxFixer(test_cases_content)
    regex = r"self\.assert(?:Equal|AlmostEqual)\(function_put\((.*?), (.*?), (.*?)\), (.*?)\)"
    matches = re.findall(regex, fixed_syntax)

    syntax_fixed_IUT = []

    for match in matches:
        input_1 = match[0]
        input_2 = match[1]
        input_3 = match[2]
        expected_value = match[3]

        if expected_value.count('(') > expected_value.count(')'):
            expected_value += ')'

        inputs = (eval(input_1.strip()), eval(input_2.strip()), eval(input_3.strip()))
        output = eval(expected_value.strip())

        syntax_fixed_IUT.append({
            'input': inputs,
            'output': output
        })

    exec(put_code_content, globals())
    PUT = globals().get("function_put")

    intended_behavior_fixed = IntendedBehaviorFixer(syntax_fixed_IUT, PUT)
    updated_test_code = fixed_syntax

    for test_case in intended_behavior_fixed:
        a, b, c = test_case['input']
        new_output = test_case['output']

        pattern = rf"self\.assert(?:Equal|AlmostEqual)\(add\({a}, {b}, {c}\), (.*?)\)"
        replacement = f"self.assert{'AlmostEqual' if isinstance(new_output, float) else 'Equal'}(add({a}, {b}), {new_output})"

        updated_test_code = re.sub(pattern, replacement, updated_test_code)

    os.makedirs("Final_Test_Suits_Few_shot", exist_ok=True)
    test_suit_path = f"./Final_Test_Suits_Few_shot/test_put_code_{put_code_file_name}"
    
    with open(test_suit_path, 'w') as f:
        f.write(updated_test_code)

    print("Test cases updated and saved!")
    
    return updated_test_code



In [103]:
updated_test_code = fix_and_generate_tests(test_cases_content, put_code_content, put_code_file_name)

Test cases updated and saved!


In [104]:
goals_prompt = f"""
You are an expert Python mutation testing engineer. 
Your task is to generate as many different mutants as possible for the given Python code. 
Each mutant must make only one small change (like changing + to -, > to >=, etc.) 
but the code should stay valid Python syntax.

Here is the original code:

{put_code_content}

Respond with all the mutated versions of the code. 
Label each mutant as: # Mutant 1, # Mutant 2, and so on.

Do NOT explain anything. Just show the mutated code blocks.
CHECK: not generating equivalent mutants.

Original code:

def add(a, b):
    return a + b
    
Generated Mutants:

#Mutant 1:
def add(a, b):
    return a - b
    
#Mutant 2:
def add(a, b):
    return a * b
    
#Mutant 3:
def add(a, b):
    return a / b
    
"""
mutant_code = llm.generate_content(goals_prompt)

print(mutant_code.text)


```python
# Mutant 1
def function_put(a, b, c) -> str:
    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):
        raise TypeError("All inputs must be integers (bool is not allowed).")
    elif not all(x >= 0 for x in [a, b, c]):
        raise ValueError("All sides must be greater than zero.")
    elif a + b <= c or a + c <= b or b + c <= a:
        return "Not a valid triangle"
    elif a == b == c:
        return "Equilateral"
    elif a == b or b == c or a == c:
        return "Isosceles"
    return "Scalene"

# Mutant 2
def function_put(a, b, c) -> str:
    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):
        raise TypeError("All inputs must be integers (bool is not allowed).")
    elif not all(x > 0 for x in [a, b, c]):
        raise ValueError("All sides must be greater than zero.")
    elif a + b < c or a + c <= b or b + c <= a:
        return "Not a valid triangle"
    elif a == b == c:
        return "Equilateral

In [105]:
mutants_array = mutant_code.text.strip()
mutants_array = mutants_array.replace("```python", "").replace("```", "").strip()

mutants_array = mutants_array.split("\n\n")
mutants_array = [mutant.strip() for mutant in mutants_array if mutant.strip()]

mutants_array = [mutant.split("\n", 1)[-1].strip() for mutant in mutants_array]

print(mutants_array)


['def function_put(a, b, c) -> str:\n    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):\n        raise TypeError("All inputs must be integers (bool is not allowed).")\n    elif not all(x >= 0 for x in [a, b, c]):\n        raise ValueError("All sides must be greater than zero.")\n    elif a + b <= c or a + c <= b or b + c <= a:\n        return "Not a valid triangle"\n    elif a == b == c:\n        return "Equilateral"\n    elif a == b or b == c or a == c:\n        return "Isosceles"\n    return "Scalene"', 'def function_put(a, b, c) -> str:\n    if not all(isinstance(x, int) and not isinstance(x, bool) for x in [a, b, c]):\n        raise TypeError("All inputs must be integers (bool is not allowed).")\n    elif not all(x > 0 for x in [a, b, c]):\n        raise ValueError("All sides must be greater than zero.")\n    elif a + b < c or a + c <= b or b + c <= a:\n        return "Not a valid triangle"\n    elif a == b == c:\n        return "Equilateral"\n    el

In [106]:
mutants_folder = "Mutants_Few_shot"
subfolder_name = f"PUT-{code_number}"
subfolder_path = os.path.join(mutants_folder, subfolder_name)

if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path) 

for i, mutant in enumerate(mutants_array, start=1):
    file_name = f"Mutants_Few_shot/PUT-{code_number}/mutant_1_{i}.py"  
    
    with open(file_name, "w") as f:
        f.write(mutant)

print("Mutants saved successfully!")

Mutants saved successfully!


In [107]:
pos = updated_test_code.find("import unittest")
test_case_only = updated_test_code[pos + len("import unittest"):].lstrip()

In [108]:
for i, mutant in enumerate(mutants_array, start=1):
    test_case_file_name = f"Mutants_Few_shot/PUT-{code_number}/mutant_1_{i}_test_case.py"
    with open(test_case_file_name, "w") as f:
        f.write("import unittest\n")
        f.write(mutant + "\n\n" + test_case_only)

print("Test case files for all mutants generated successfully!")


Test case files for all mutants generated successfully!


In [109]:
import subprocess
import unittest
import os

def Mutation_Testing():
    killed_mutants = 0
    survived_mutants = []
    mutant_test_directory = f'Mutants_Few_shot/PUT-{code_number}' 
    
    if not os.path.exists(mutant_test_directory):
        print(f"Directory '{mutant_test_directory}' not found.")
        return 0, survived_mutants

    mutant_files = [f for f in os.listdir(mutant_test_directory) if f.startswith('mutant_') and f.endswith('_test_case.py')]

    if len(mutant_files) == 0:
        print("No mutant test files found.")
        return 0, survived_mutants
    
    for mutant_file in mutant_files:
        try:
            result = subprocess.run(
                ["python", "-m", "unittest", os.path.join(mutant_test_directory, mutant_file)],
                capture_output=True,
                text=True
            )
            if result.returncode == 0:
                survived_mutants.append(mutant_file)
                print(f"Mutant {mutant_file} survived.")
            else:
                killed_mutants += 1
                print(f"Mutant {mutant_file} killed.")
        except Exception as e:
            print(f"Error running tests for {mutant_file}: {e}")

    total_mutants = len(mutant_files)
    mutation_score = (killed_mutants / total_mutants) * 100

    return mutation_score, survived_mutants

MS, survived_mutants = Mutation_Testing()

print(f"Mutation Score: {MS:.2f}%")
print(f"Survived Mutants: {survived_mutants}")


Mutant mutant_1_10_test_case.py killed.
Mutant mutant_1_11_test_case.py survived.
Mutant mutant_1_12_test_case.py killed.
Mutant mutant_1_1_test_case.py survived.
Mutant mutant_1_2_test_case.py survived.
Mutant mutant_1_3_test_case.py survived.
Mutant mutant_1_4_test_case.py killed.
Mutant mutant_1_5_test_case.py killed.
Mutant mutant_1_6_test_case.py killed.
Mutant mutant_1_7_test_case.py survived.
Mutant mutant_1_8_test_case.py survived.
Mutant mutant_1_9_test_case.py killed.
Mutation Score: 50.00%
Survived Mutants: ['mutant_1_11_test_case.py', 'mutant_1_1_test_case.py', 'mutant_1_2_test_case.py', 'mutant_1_3_test_case.py', 'mutant_1_7_test_case.py', 'mutant_1_8_test_case.py']


In [110]:
survived_mutants

['mutant_1_11_test_case.py',
 'mutant_1_1_test_case.py',
 'mutant_1_2_test_case.py',
 'mutant_1_3_test_case.py',
 'mutant_1_7_test_case.py',
 'mutant_1_8_test_case.py']

In [111]:
def AugmentingPrompt(initial_prompt, mutant_info, test_cases_content):
    instruction = f""" 
    # Mutant to kill:
        {mutant_info}

    # Current test cases:
    
    {test_cases_content}

    # Add exactly One or Two new test case to kill the given mutant. Do not repeat or modify existing tests.
"""

    return f"{initial_prompt}\n{instruction}"


In [112]:
goals_prompt_f2 = f"""
You are an expert Python software engineer.

You are given a Python file with existing `unittest` test cases.

Your goals:
- Combine all existing test cases into a single method called `test_all_cases`.
- Do NOT change or regenerate existing test cases.
- ##Append New ONE OR TWO Test-case## of the test content function So KILL One MUTANT.
- Use `assertEqual`, `assertTrue`, `assertFalse`, etc., with meaningful values. 
- Output only the full Python `unittest` code, including the existing test cases + exactly one new test.
- #OUTPUT not include put_code_content or any other text. Just the test cases.
- Do not include any explanations or comments in the output.
- Generated Test cases ouput should be valid in Python code.
Here is the original code under test:

```python
{put_code_content}

Example 1:
Code to write test cases for:

```python
def add(a, b):
    return a + b
"""

In [113]:
test_suit_path = f"./Final_Test_Suits_Few_shot/test_put_code_{put_code_file_name}"

while MS < 100 and survived_mutants:
     
    if survived_mutants:
        mutant = survived_mutants.pop()
        print(f"Survived mutant: {mutant}")
    else:
        print("No more survived mutants to process.")
        break
    
    
    augmenting_prompt = AugmentingPrompt(goals_prompt_f2, mutant, test_cases_content)  
    new_added_test_case_response = LLMC(augmenting_prompt)  
    new_added_test_case = new_added_test_case_response.strip()
    new_added_test_case = new_added_test_case.replace("```python", "").replace("```", "").strip()
    
    print("------------------------------------------------------------------------------------------")    
    print(new_added_test_case)
    print("------------------------------------------------------------------------------------------")
    with open(test_suit_path, 'w') as f:   
            # f.write(put_code_content)
            f.write(new_added_test_case)  
            
    with open(test_suit_path, 'r') as f:
        test_cases_content = f.read()
        
    fix_and_generate_tests(test_cases_content, put_code_content, put_code_file_name)
    
            
    pos = new_added_test_case.find("class")
    test_case_only = new_added_test_case[pos:]
   
    for i, mutant in enumerate(mutants_array, start=1):
        test_case_file_name = f"Mutants_Few_shot/PUT-{code_number}/mutant_1_{i}_test_case.py"
        with open(test_case_file_name, "w") as f:
            f.write("import unittest\n")
            f.write(mutant + "\n\n" + test_case_only)
             
    New_MS, New_survived_mutants = Mutation_Testing()
    
    print(f"New Mutation Score: {New_MS:.2f}%")
    print(f"New Survived Mutants: {New_survived_mutants}")
    if(New_MS == 100):
        print("All mutants killed!")
        break 
    
    if(New_MS > MS):
        MS = New_MS
        survived_mutants = New_survived_mutants
        
        

    print(f"Mutation Score: {MS:.2f}%")
    print(f"Survived Mutants: {survived_mutants}")

Survived mutant: mutant_1_8_test_case.py
------------------------------------------------------------------------------------------
import unittest

class TestFunctionPut(unittest.TestCase):
    def test_all_cases(self):
        self.assertEqual(function_put(2, 2, 2), "Equilateral")
        self.assertEqual(function_put(3, 4, 5), "Scalene")
        self.assertEqual(function_put(2, 3, 2), "Isosceles")
        self.assertRaises(ValueError, function_put, 1, 2, -1)
------------------------------------------------------------------------------------------
Test cases updated and saved!
Mutant mutant_1_10_test_case.py killed.
Mutant mutant_1_11_test_case.py survived.
Mutant mutant_1_12_test_case.py killed.
Mutant mutant_1_1_test_case.py survived.
Mutant mutant_1_2_test_case.py survived.
Mutant mutant_1_3_test_case.py survived.
Mutant mutant_1_4_test_case.py killed.
Mutant mutant_1_5_test_case.py killed.
Mutant mutant_1_6_test_case.py killed.
Mutant mutant_1_7_test_case.py survived.
Mutant mut